In [22]:
from decimal import Decimal
import json
import psycopg2

mid_lat, mid_long = 36.00192134954784, -78.937821464194
"""
UTC,latitude,longitude,HDOP,altitude,fix,COG,spkm,spkn,date,nsat
145355.00,36.00280,-78.93724,2.1,124.7,3,131.1,0.0,0.0,201023,4
is_tdd,mcc,mnc,cellid,pcid,earfcn,freq_band_ind,ul_bandwidth,dl_bandwidth,tac,rsrp,rsrq,rssi,sinr,srxlev
"TDD",315,010,6A401,20,56140,48,5,5,2,-89,-6,-63,25,-
earfcn,pcid,srq,rsrp,rssi,sinr,srxlev
56140,20,-6,-88,-62,-,51
56640,40,-8,-91,-75,-,51
55540,7,-7,-109,-94,-,0
55540,328,-19,-129,-101,-,0
55340,6,-9,-114,-97,-,0
55340,288,-15,-121,-97,-,0
55940,288,-19,-132,-104,-,0
"""




def parse_data(src):
# Loop through each input block
    with open(src, "r") as file:
        input_data = file.read()

    # Split input by "!!!!!!!!!!!"
    inputs = input_data.strip().split("!!!!!!!!!!!\n")
    # Initialize a dictionary to store parsed data
    parsed_data = {'lat': [], 'lon': [], 'pcid': [], 'rsrp': []}
    for i, data_block in enumerate(inputs):
        # Skip empty blocks
        if not data_block:
            continue

        # Split lines in the block
        lines = data_block.strip().split('\n')
        lat_lon_line = lines[1].split(',')
        lat, lon = float(lat_lon_line[1]), float(lat_lon_line[2])

        serving_cell_line = lines[3].split(",")
        serving_pcid = int(serving_cell_line[4])
        serving_rsrp = float(serving_cell_line[10])
        parsed_data['lat'].append(lat)
        parsed_data['lon'].append(lon)
        parsed_data['pcid'].append(serving_pcid)
        parsed_data['rsrp'].append(serving_rsrp)

        for _, lll in enumerate(lines[5:]):
            neighbour_cell_line = lll.split(",")
            neighbour_pcid = int(neighbour_cell_line[1])
            neighbour_rsrp = float(neighbour_cell_line[3])

            parsed_data['lat'].append(lat)
            parsed_data['lon'].append(lon)
            parsed_data['pcid'].append(neighbour_pcid)
            parsed_data['rsrp'].append(neighbour_rsrp)
        # print(lines)
    print('# data points', len(parsed_data['rsrp']))
    return pd.DataFrame(parsed_data)


def get_data_for_this_pci(pcid, src_dict):
    ret_dict = {'lat': [], 'lon': [], 'pcid': [], 'rsrp': []}
    for idx, cur_pcid in enumerate(src_dict['pcid']):
        if pcid == cur_pcid:
            ret_dict['lat'].append(src_dict['lat'][idx])
            ret_dict['lon'].append(src_dict['lon'][idx])
            ret_dict['pcid'].append(src_dict['pcid'][idx])
            ret_dict['rsrp'].append(src_dict['rsrp'][idx])
    print(len(ret_dict['pcid']))
    return pd.DataFrame(ret_dict)


def get_unique_pci_list(src_dict):
    return set(src_dict['pcid'])


parsed_dict = parse_data("cbrs_measurement_1298270f-23e3-4afe-ba20-37dc1f53a61a.txt")
get_unique_pci_list(src_dict=parsed_dict)

# data points 4257


{3, 6, 7, 20, 40, 237, 288, 290, 312, 328}

In [10]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
from itertools import compress

load_dotenv()
API_KEY = os.getenv("API_KEY")

In [11]:
from bokeh.io import output_notebook, export_png
from bokeh.plotting import figure, show, gmap
from bokeh.models import ColumnDataSource, LogColorMapper, LinearColorMapper, HoverTool, GMapOptions, WheelZoomTool, ArrowHead
from bokeh.models import ColorBar
import bokeh.palettes as palettes
output_notebook()

Loading BokehJS ...

In [31]:
def plot_gmap_Duke_ss(mPci, inner_field, df_source: dict, mid_lat=mid_lat, mid_long=mid_long,
                      zoom=17, circle_size=4):
    # print(list(compress(df_source['longitude'], df_source['mPci'] == mPci)))
    # print(df_source['longitude'][df_source['mPci'] == mPci])
    df_source = get_data_for_this_pci(pcid=mPci, src_dict=df_source)
    df_source = pd.DataFrame(list(
        zip(*[list(compress(df_source[kkey], df_source['pcid'] == mPci))for kkey in list(df_source)])
    ),
                             columns=list(df_source))
    print(df_source.head())
    print('number of data points', len(df_source['lon']))
    inner_gmap_options = GMapOptions(
    lat=np.mean(df_source['lat']),
    lng=np.mean(df_source['lon']),
    map_type='satellite',
    zoom=zoom
    )
    inner_hover = HoverTool(
        tooltips=[
            (inner_field, '@' + inner_field + '{0.0}'),
        ]
    )

    inner_p = gmap(
        API_KEY,
        inner_gmap_options,
        title=str(inner_field + ' mPci: ' + str(mPci)),
        width=800,
        height=600,
        tools=[inner_hover, 'pan']
    )
    inner_source = ColumnDataSource(df_source)

    inner_color_mapper = LinearColorMapper(palette=palettes.inferno(20))
    inner_p.circle('lon', 'lat', source=inner_source,
              color={'field': inner_field, 'transform': inner_color_mapper},
              size=circle_size)
    inner_color_bar = ColorBar(color_mapper=inner_color_mapper, location=(0, 0))
    inner_p.add_layout(inner_color_bar, 'right')
    inner_p.add_tools(WheelZoomTool())
    return inner_p


p1 = plot_gmap_Duke_ss(mPci=int(237), inner_field='rsrp',
                         df_source=parsed_dict, circle_size=7)
print(get_unique_pci_list(src_dict=parsed_dict))
show(p1)

19
        lat       lon  pcid   rsrp
0  36.00136 -78.93847   237 -114.0
1  36.00137 -78.93846   237 -114.0
2  36.00139 -78.93845   237 -114.0
3  36.00140 -78.93845   237 -114.0
4  36.00140 -78.93845   237 -114.0
number of data points 19
{288, 290, 3, 6, 7, 40, 328, 237, 20, 312}


In [18]:
dict_test = {'latitude': [mid_lat, mid_lat + 0.0001], 'longitude': [mid_long, mid_long + 0.0001],
             'rssi': [-40, -50]}
df_test = pd.DataFrame.from_dict(dict_test)
p2 = plot_gmap_Duke_ss(inner_field='rssi',
                         title='rssi SigCapDetails',
                         df_source=df_test, circle_size=7)
show(p2)

   longitude   latitude  rssi
0 -78.937821  36.001921   -40
1 -78.937721  36.002021   -50


In [6]:
from bokeh.models import (Arrow, Label, NormalHead, OpenHead,
                          Plot, Range1d, TeeHead, VeeHead)
from bokeh.plotting import show

ARROW_HEADS = [VeeHead]#[TeeHead, OpenHead, NormalHead, VeeHead]
HEIGHT = 35 * len(ARROW_HEADS)

p = Plot(width=150, height=HEIGHT,
         x_range=Range1d(0,1), y_range=Range1d(-0.5, len(ARROW_HEADS) - 0.5),
         toolbar_location=None, outline_line_color=None, min_border_left=0,
         min_border_right=0, min_border_top=0, min_border_bottom=0)

for i, style in enumerate(ARROW_HEADS):
    arrow = Arrow(x_start=0.2, y_start=i, x_end=0.2, y_end=i-0.1, end=style(
        line_width=2, line_color='blue', line_alpha=0.8,
                             size=10,
                             line_dash='solid'
    ))
    p.add_layout(arrow)


show(p)

In [7]:
def plot_gmap_Duke_ss_arrow_head(mPci, inner_field='rssi', df_source=df_SigCapDetails,
                                 mid_lat=mid_lat, mid_long=mid_long,
                      zoom=17, circle_size=4, ang_sep=10):
    # df_source = pd.DataFrame(list(zip(*[df_source[kkey] for kkey in list(df_source)])),
    #                          columns=list(df_source))
    df_source = pd.DataFrame(list(zip(#df_source['uid'],
        list(compress(df_source['longitude'], df_source['mPci'] == mPci)),
        list(compress(df_source['latitude'], df_source['mPci'] == mPci)),
        list(compress(df_source['rssi'], df_source['mPci'] == mPci)),
        list(compress(df_source['azimuth'], df_source['mPci'] == mPci)),
        list(compress(df_source['mPci'], df_source['mPci'] == mPci)),
        np.zeros(len(list(compress(df_source['azimuth'], df_source['mPci'] == mPci)))),
        np.zeros(len(list(compress(df_source['azimuth'], df_source['mPci'] == mPci))))
                                      #np.pi/2 - (np.radians(df_source['azimuth']) + ang_sep/2),
                                      #np.pi/2 - (np.radians(df_source['azimuth']) - ang_sep/2)
                                      )
                                  ),
                             columns=[#'uid',
                                      'longitude', 'latitude', 'rssi', 'azimuth', 'mPci', 'azimuth_start',
                                      'azimuth_end'])
    # print(df_source.head())
    # converting from
    for idx, angle in enumerate(df_source['azimuth']):
        if -90 < df_source['azimuth'][idx] < 180:
            df_source['azimuth_end'][idx] = 90 - ((df_source['azimuth'][idx]) - ang_sep/2)
            df_source['azimuth_start'][idx] = 90 - ((df_source['azimuth'][idx]) + ang_sep/2)
        if -180 < df_source['azimuth'][idx] < -90:
            df_source['azimuth_end'][idx] = -270 - ((df_source['azimuth'][idx]) - ang_sep/2)
            df_source['azimuth_start'][idx] = -270 - ((df_source['azimuth'][idx]) + ang_sep/2)

    # print(df_source.head())
    inner_gmap_options = GMapOptions(
    lat=np.mean(df_source['latitude']),
    lng=np.mean(df_source['longitude']),
    map_type='satellite',
    zoom=zoom
    )
    inner_hover = HoverTool(
        tooltips=[
            (inner_field, '@' + inner_field + '{0.0}'),
            #('uid', '@' + 'uid' + '{0.0}')
            ('azimuth', '@azimuth{0.0}')
        ]
    )

    inner_p = gmap(
        API_KEY,
        inner_gmap_options,
        title=inner_field + '; mPci: ' + str(mPci) + '; num data points: ' + str(len(df_source['azimuth'])),
        width=800,
        height=600,
        tools=[inner_hover, 'pan']
    )

    inner_source = ColumnDataSource(df_source)

    inner_color_mapper = LinearColorMapper(palette=palettes.inferno(20))

    inner_p.annular_wedge(x='longitude', y='latitude', inner_radius=0, outer_radius=20,
                          start_angle='azimuth_start', end_angle='azimuth_end', source=inner_source,
                          color={'field': inner_field, 'transform': inner_color_mapper}, alpha=0.8,
                          start_angle_units='deg', end_angle_units='deg')
    inner_color_bar = ColorBar(color_mapper=inner_color_mapper, location=(0, 0))
    inner_p.add_layout(inner_color_bar, 'right')
    inner_p.add_tools(WheelZoomTool())
    return inner_p

# print(df_SigCapDetails['longitude'])
# export_png(obj=plot_gmap_Duke_ss_arrow_head(mPci=20), filename='20.png')
show(plot_gmap_Duke_ss_arrow_head(mPci=mPci_keys[9]))

/var/folders/sd/_4gcyf1x307dwdn_5_g9jqy80000gn/T/ipykernel_23009/2153484946.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_source['azimuth_end'][idx] = 90 - ((df_source['azimuth'][idx]) - ang_sep/2)
/var/folders/sd/_4gcyf1x307dwdn_5_g9jqy80000gn/T/ipykernel_23009/2153484946.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_source['azimuth_start'][idx] = 90 - ((df_source['azimuth'][idx]) + ang_sep/2)
/var/folders/sd/_4gcyf1x307dwdn_5_g9jqy80000gn/T/ipykernel_23009/2153484946.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentat